In [4]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import KFold
from sklearn import cross_validation

"""Importing Data"""
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")
print type(df_train)

"""Dividing male and female data"""
df_male_train = df_train[df_train['Sex'] == 'male']
print type(df_male_train)
df_female_train = df_train[df_train['Sex'] == 'female']

df_male_test = df_test[df_test['Sex'] == 'male']
df_female_test = df_test[df_test['Sex'] == 'female']

"""Processing Datasets"""
def predict (df_train, df_test):
       
    # df = df.drop(['PassengerId','Name','Ticket'], axis=1)
    
    df_train['Age'] = df_train['Age'].fillna(df_train['Age'].median())
    df_train['Fare'] = df_train['Fare'].fillna(df_train['Fare'].median())

    df_test['Age'] = df_test['Age'].fillna(df_test['Age'].median())
    df_test['Fare'] = df_test['Fare'].fillna(df_test['Fare'].median())
    
    """
    #Sex   male: 0, female: 1df[]
    df_train.loc[df_train['Sex'] == 'male', 'Sex'] = 0
    df_train.loc[df_train['Sex'] == 'female', 'Sex'] = 1

    df_test.loc[df_test['Sex'] == 'male', 'Sex'] = 0
    df_test.loc[df_test['Sex'] == 'female', 'Sex'] = 1
    """
    
    df_train['Embarked_S'] = df_train['Embarked']
    df_train['Embarked_C'] = df_train['Embarked']
    df_train['Embarked_Q'] = df_train['Embarked']
    
    df_test['Embarked_S'] = df_test['Embarked']
    df_test['Embarked_C'] = df_test['Embarked']
    df_test['Embarked_Q'] = df_test['Embarked']
    
    #Embarked_S   Southampton: 1, rest: 0
    df_train['Embarked_S'] = df_train['Embarked'].fillna('S')
    df_test['Embarked_S'] = df_test['Embarked'].fillna('S')
    
    df_train.loc[df_train["Embarked_S"] == 'S', "Embarked_S"] =1
    df_train.loc[df_train["Embarked_S"] == 'C', "Embarked_S"] =0
    df_train.loc[df_train["Embarked_S"] == 'Q', "Embarked_S"] =0

    df_test.loc[df_test["Embarked_S"] == 'S', "Embarked_S"] =1
    df_test.loc[df_test["Embarked_S"] == 'C', "Embarked_S"] =0
    df_test.loc[df_test["Embarked_S"] == 'Q', "Embarked_S"] =0

    #Embarked_C   Cherbourg: 1, rest: 0
    df_train['Embarked_C'] = df_train['Embarked'].fillna('C')
    df_test['Embarked_C'] = df_test['Embarked'].fillna('C')

    df_train.loc[df_train["Embarked_C"] == 'S', "Embarked_C"] =0
    df_train.loc[df_train["Embarked_C"] == 'C', "Embarked_C"] =1
    df_train.loc[df_train["Embarked_C"] == 'Q', "Embarked_C"] =0

    df_test.loc[df_test["Embarked_C"] == 'S', "Embarked_C"] =0
    df_test.loc[df_test["Embarked_C"] == 'C', "Embarked_C"] =1
    df_test.loc[df_test["Embarked_C"] == 'Q', "Embarked_C"] =0

    #Embarked_Q   Queestown: 1, rest: 0
    df_train['Embarked_Q'] = df_train['Embarked'].fillna('Q')
    df_test['Embarked_Q'] = df_test['Embarked'].fillna('Q')

    df_train.loc[df_train["Embarked_Q"] == 'S', "Embarked_Q"] =0
    df_train.loc[df_train["Embarked_Q"] == 'C', "Embarked_Q"] =0
    df_train.loc[df_train["Embarked_Q"] == 'Q', "Embarked_Q"] =1

    df_test.loc[df_test["Embarked_Q"] == 'S', "Embarked_Q"] =0
    df_test.loc[df_test["Embarked_Q"] == 'C', "Embarked_Q"] =0
    df_test.loc[df_test["Embarked_Q"] == 'Q', "Embarked_Q"] =1

    predictors = ["Pclass", "Age", "SibSp", "Fare", "Parch", "Embarked_S", "Embarked_C", "Embarked_Q"]

    alg = LinearRegression()
    kf = KFold(df_train.shape[0], n_folds=3)

    predictions = []
    for train, test in kf:
        train_predictors = (df_train[predictors].iloc[train,:])
        train_target = df_train["Survived"].iloc[train]
        alg.fit(train_predictors, train_target)
        test_predictions = alg.predict(df_train[predictors].iloc[test,:])
        predictions.append(test_predictions)
    
    alg = LogisticRegression()
    scores = cross_validation.cross_val_score(alg, df_train[predictors], df_train["Survived"], cv=3)
    print(scores.mean())

    alg.fit(df_train[predictors], df_train["Survived"])
    predictions = alg.predict(df_test[predictors])
    submission = pd.DataFrame({
            "PassengerId": df_test["PassengerId"],
            "Survived": predictions
            })
    
    return submission

male_submission = predict (df_male_train, df_male_test)
female_submission = predict (df_female_train, df_female_test)
frames = [male_submission, female_submission]
submission = pd.concat(frames)

submission.to_csv('titanic.csv', index=False)


/home/jkim/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/jkim/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/jkim/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/in

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
0.809378598158
0.770787545788


/home/jkim/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


I forgot .loc the embarked port to value 0, 1 and 2

1st revision Score: 0.74641

Divided Embarked columns into 3: Embarked_S with value 1 for passengers who embarked in port S and 0 for rest,
                                 Embarked_C with value 1 for passengers who embarked in port C and 0 for rest,
                                 Embarked_Q with value 1 for passengers who embarked in port Q and 0 for rest

Tried using RandomForestClassifier instead of LogisticRegression

3rd revision Score: 0.78947

predicted for male and female seperately and combined them afterwards